# Distribuciones conjuntas mediante procesamiento automático por lotes: un sencillo tutorial

##### Copyright 2020 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/probability/examples/JointDistributionAutoBatched_A_Gentle_Tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/probability/examples/JointDistributionAutoBatched_A_Gentle_Tutorial.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/probability/examples/JointDistributionAutoBatched_A_Gentle_Tutorial.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/probability/examples/JointDistributionAutoBatched_A_Gentle_Tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table>

### Introducción

TensorFlow Probability (TFP) ofrece una serie de abstracciones `JointDistribution` que facilitan la inferencia probabilística al permitir al usuario expresar fácilmente un modelo gráfico probabilístico en una forma casi matemática; la abstracción genera métodos para tomar muestras del modelo y evaluar la probabilidad logarítmica de las muestras del modelo. En este tutorial, revisamos variantes de "procesamiento automático por lotes", que se desarrollaron después de las abstracciones originales `JointDistribution`. En comparación con las abstracciones originales, sin procesamiento automático por lotes, las versiones con procesamiento automático por lotes son más sencillas de usar y más ergonómicas, lo que permite expresar muchos modelos con menos texto repetitivo. En esta colaboración, exploramos un modelo simple con (quizás tedioso) detalle, dejando en claro los problemas que resuelve el procesamiento automático por lotes y (con suerte) enseñando al lector más sobre los conceptos de forma de TFP a lo largo del camino.

Antes de la introducción del procesamiento automático por lotes, existían algunas variantes diferentes de `JointDistribution`, correspondientes a diferentes estilos sintácticos para expresar modelos probabilísticos: `JointDistributionSequential`, `JointDistributionNamed` y `JointDistributionCoroutine`. El procesamiento automático por lotes existe como un mixin, por lo que ahora tenemos variantes `AutoBatched` de todos estos. En este tutorial, exploramos las diferencias entre `JointDistributionSequential` y `JointDistributionSequentialAutoBatched`; sin embargo, todo lo que hacemos aquí se puede aplicar a las otras variantes esencialmente sin cambios.


### Dependencias y requisitos previos


In [ ]:
#@title Import and set ups{ display-mode: "form" }

import functools
import numpy as np

import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()

import tensorflow_probability as tfp

tfd = tfp.distributions

### Requisito previo: un problema de regresión bayesiana

Veamos un ejemplo sencillo de regresión bayesiana:

$$
\begin{align*}
m & \sim \text{Normal}(0, 1) \\
b & \sim \text{Normal}(0, 1) \\
Y & \sim \text{Normal}(mX + b, 1)
\end{align*}
$$

En este modelo, `m` y `b` se extraen de normales estándar, y las observaciones `Y` se extraen de una distribución normal cuya media depende de las variables aleatorias `m` y `b` de algunas covariables (no aleatorias) `X` (Para simplificar, en este ejemplo asumimos que se conoce la escala de todas las variables aleatorias).

Para realizar inferencias en este modelo, necesitaríamos conocer las covariables `X` y las observaciones `Y`, pero para los fines de este tutorial, solo necesitaremos `X`, por lo que definimos una `X` ficticia simple:

In [ ]:
X = np.arange(7)
X

array([0, 1, 2, 3, 4, 5, 6])

### Desiderata

En la inferencia probabilística, a menudo queremos realizar dos operaciones básicas:

- `sample`: extraer muestras del modelo.
- `log_prob`: calcular la probabilidad logarítmica de una muestra del modelo.

La contribución clave de las abstracciones `JointDistribution` de TFP (así como de muchos otros enfoques de programación probabilística) es que permite a los usuarios escribir un modelo *una vez* y acceder a cálculos `sample` y `log_prob`.

Teniendo en cuenta que tenemos 7 puntos en nuestro conjunto de datos (`X.shape = (7,)`), ahora podemos establecer los desiderata para una excelente `JointDistribution`:

- `sample()` debería producir una lista de `Tensors` que tienen forma `[(), (), (7,)`], correspondientes a la pendiente escalar, el sesgo escalar y las observaciones vectoriales, respectivamente.
- `log_prob(sample())` debería producir un escalar: la probabilidad logarítmica de una pendiente, sesgo y observaciones particulares.
- `sample([5, 3])` debería producir una lista de `Tensors` que tengan forma `[(5, 3), (5, 3), (5, 3, 7)]`, representando un `(5, 3)` - *lote* de muestras del modelo.
- `log_prob(sample([5, 3]))` debería producir un `Tensor` con forma (5, 3).

Ahora veremos una sucesión de modelos `JointDistribution`, veremos cómo alcanzar los desiderata anteriores y, con suerte, aprenderemos un poco más sobre las formas de TFP a lo largo del proceso.

Alerta de spoiler: el enfoque que satisface los desiderata anteriores sin agregar elementos repetitivos es el [procesamiento automático por lotes](#scrollTo=_h7sJ2bkfOS7). 

### Primer intento; `JointDistributionSequential`

In [ ]:
jds = tfd.JointDistributionSequential([
    tfd.Normal(loc=0., scale=1.),   # m
    tfd.Normal(loc=0., scale=1.),   # b
    lambda b, m: tfd.Normal(loc=m*X + b, scale=1.) # Y
])

Esto es prácticamente una traducción directa del modelo al código. La pendiente `m` y el sesgo `b` son sencillos. `Y` se define con una función `lambda`: el patrón general es que una función `lambda` de $k$ argumentos en un `JointDistributionSequential` (JDS) utiliza las distribuciones $k$ anteriores en el modelo. Nótese que el orden es "inverso".

Llamaremos `sample_distributions`, que devuelve tanto una muestra *como* las "subdistribuciones" subyacentes que se utilizaron para generar la muestra. (Podríamos haber producido solo la muestra llamando a `sample`; más adelante en el tutorial será conveniente tener también las distribuciones). La muestra que se produjo está bien:

In [ ]:
dists, sample = jds.sample_distributions()
sample

[<tf.Tensor: shape=(), dtype=float32, numpy=-1.668757>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.6585061>,
 <tf.Tensor: shape=(7,), dtype=float32, numpy=
 array([ 0.18573815, -1.79962   , -1.8106272 , -3.5971394 , -6.6625295 ,
        -7.308844  , -9.832693  ], dtype=float32)>]

Pero `log_prob` produce un resultado con una forma no deseada:

In [ ]:
jds.log_prob(sample)

<tf.Tensor: shape=(7,), dtype=float32, numpy=
array([-4.4777603, -4.6775575, -4.7430477, -4.647725 , -4.5746684,
       -4.4368567, -4.480562 ], dtype=float32)>

Y la multiplicidad de muestras no funciona:

In [ ]:
try:
  jds.sample([5, 3])
except tf.errors.InvalidArgumentError as e:
  print(e)

Incompatible shapes: [5,3] vs. [7] [Op:Mul]


Intentemos entender qué está pasando.

### Una breve reseña: forma de lote y de evento

En TFP, una distribución de probabilidad ordinaria (no una `JointDistribution`) tiene una *forma de evento* y una *forma de lote*, y comprender la diferencia es crucial para usar TFP de forma eficiente:

- La forma de evento describe la forma de una única extracción de la distribución; la extracción puede depender de todas las dimensiones. Para distribuciones escalares, la forma de evento es []. Para una MultivariateNormal de 5 dimensiones, la forma de evento es [5].
- La forma de lote describe extracciones independientes, no distribuidas de manera idéntica, también conocida como "lote" de distribuciones. Representar un lote de distribuciones en un único objeto Python es una de las formas clave en que TFP logra eficiencia a escala.

Para nuestros propósitos, un hecho crítico a tener en cuenta es que si llamamos `log_prob` en una sola muestra de una distribución, el resultado siempre tendrá una forma que coincida (es decir, que tiene como dimensiones más a la derecha) la forma de *lote*.

Para obtener una descripción más detallada de las formas, consulte el [tutorial "Explicación de las formas de distribuciones de TensorFlow"](https://www.tensorflow.org/probability/examples/Understanding_TensorFlow_Distributions_Shapes).


### ¿Por qué `log_prob(sample())` no produce un escalar? 

Usemos nuestro conocimiento sobre la forma de lote y de evento para explorar qué está sucediendo con `log_prob(sample())`. Aquí está nuestra muestra nuevamente:

In [ ]:
sample

[<tf.Tensor: shape=(), dtype=float32, numpy=-1.668757>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.6585061>,
 <tf.Tensor: shape=(7,), dtype=float32, numpy=
 array([ 0.18573815, -1.79962   , -1.8106272 , -3.5971394 , -6.6625295 ,
        -7.308844  , -9.832693  ], dtype=float32)>]

Y aquí están nuestras distribuciones:

In [ ]:
dists

[<tfp.distributions.Normal 'Normal' batch_shape=[] event_shape=[] dtype=float32>,
 <tfp.distributions.Normal 'Normal' batch_shape=[] event_shape=[] dtype=float32>,
 <tfp.distributions.Normal 'JointDistributionSequential_sample_distributions_Normal' batch_shape=[7] event_shape=[] dtype=float32>]

La probabilidad logarítmica se calcula mediante la suma de las probabilidades logarítmicas de las subdistribuciones en los elementos (coincidentes) de las partes:

In [ ]:
log_prob_parts = [dist.log_prob(s) for (dist, s) in zip(dists, sample)]
log_prob_parts

[<tf.Tensor: shape=(), dtype=float32, numpy=-2.3113134>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-1.1357536>,
 <tf.Tensor: shape=(7,), dtype=float32, numpy=
 array([-1.0306933, -1.2304904, -1.2959809, -1.200658 , -1.1276014,
        -0.9897899, -1.0334952], dtype=float32)>]

In [ ]:
sum(log_prob_parts) - jds.log_prob(sample)

<tf.Tensor: shape=(7,), dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0.], dtype=float32)>

Entonces, un nivel de explicación es que el cálculo de probabilidad logarítmica devuelve un tensor 7 porque el tercer subcomponente de `log_prob_parts` es un tensor 7. ¿Pero por qué?

Bueno, vemos que el último elemento de `dists`, que corresponde a nuestra distribución sobre `Y` en la formulación matemática, tiene una `batch_shape` de `[7]`. En otras palabras, nuestra distribución sobre `Y` es un lote de 7 normales independientes (con medias diferentes y, en este caso, la misma escala).

Ahora entendemos lo que está mal: en JDS, la distribución sobre `Y` tiene `batch_shape=[7]`, una muestra de JDS representa escalares `b` `m` "lote" de 7 normales independientes, y `log_prob` calcula 7 probabilidades logarítmicas separadas, cada una de las cuales representa la probabilidad logarítmica de extraer `m` y `b`, y una sola observación `Y[i]` en algún `X[i]`.

### Cómo arreglar `log_prob(sample())` con `Independent`

Recuerde que `dists[2]` tiene `event_shape=[]` y `batch_shape=[7]`:

In [ ]:
dists[2]

<tfp.distributions.Normal 'JointDistributionSequential_sample_distributions_Normal' batch_shape=[7] event_shape=[] dtype=float32>

Al utilizar la metadistribución `Independent` de TFP, que convierte dimensiones de lote en dimensiones de evento, podemos convertir esto en una distribución con `event_shape=[7]` y `batch_shape=[]` (le cambiaremos el nombre `y_dist_i` porque es una distribución en `Y`, con `_i` en pie para nuestro envoltorio `Independent`): 

In [ ]:
y_dist_i = tfd.Independent(dists[2], reinterpreted_batch_ndims=1)
y_dist_i

<tfp.distributions.Independent 'IndependentJointDistributionSequential_sample_distributions_Normal' batch_shape=[] event_shape=[7] dtype=float32>

Ahora, la `log_prob` de un vector 7 es un escalar:

In [ ]:
y_dist_i.log_prob(sample[2])

<tf.Tensor: shape=(), dtype=float32, numpy=-7.9087086>

A nivel interno, `Independent` suma sobre el lote:

In [ ]:
y_dist_i.log_prob(sample[2]) - tf.reduce_sum(dists[2].log_prob(sample[2]))

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

Y, de hecho, podemos usar esto para construir un nuevo `jds_i` (la `i` nuevamente significa `Independent`) donde `log_prob` devuelve un escalar:

In [ ]:
jds_i = tfd.JointDistributionSequential([
    tfd.Normal(loc=0., scale=1.),   # m
    tfd.Normal(loc=0., scale=1.),   # b
    lambda b, m: tfd.Independent(   # Y
        tfd.Normal(loc=m*X + b, scale=1.),
        reinterpreted_batch_ndims=1)
])

jds_i.log_prob(sample)

<tf.Tensor: shape=(), dtype=float32, numpy=-11.355776>

Un par de notas:

- `jds_i.log_prob(s)` *no* es lo mismo que `tf.reduce_sum(jds.log_prob(s))`. El primero produce la probabilidad logarítmica "correcta" de la distribución conjunta. Este último suma sobre un tensor 7, cada elemento del cual es la suma de la probabilidad logarítmica de `m`, `b` y un solo elemento de la probabilidad logarítmica de `Y`, por lo que sobrecuenta `m` y `b`. (`log_prob(m) + log_prob(b) + log_prob(Y)` devuelve un resultado en lugar de generar una excepción porque TFP sigue las reglas de transmisión de TF y NumPy; agregar un escalar a un vector produce un resultado del tamaño de un vector).
- En este caso particular, podríamos haber resuelto el problema y lograr el mismo resultado usando `MultivariateNormalDiag` en lugar de `Independent(Normal(...))`. `MultivariateNormalDiag` es una distribución con valores vectoriales (es decir, ya tiene forma de evento vectorial). De hecho, `MultivariateNormalDiag` podría implementarse (pero no se implementa) como una composición de `Independent` y `Normal`. Vale la pena recordar que dado un vector `V`, las muestras de `n1 = Normal(loc=V)` y `n2 = MultivariateNormalDiag(loc=V)` son indistinguibles; la diferencia entre estas distribuciones es que `n1.log_prob(n1.sample())` es un vector y `n2.log_prob(n2.sample())` es un escalar.

### ¿Varias muestras?

Extraer varias muestras todavía no funciona:

In [ ]:
try:
  jds_i.sample([5, 3])
except tf.errors.InvalidArgumentError as e:
  print(e)

Incompatible shapes: [5,3] vs. [7] [Op:Mul]


Pensemos en por qué. Cuando llamamos `jds_i.sample([5, 3])` primero extraeremos muestras para `m` y `b`, cada una con forma `(5, 3)`. A continuación, intentaremos construir una distribución `Normal` mediante:

```
tfd.Normal(loc=m*X + b, scale=1.)
```

Pero si `m` tiene forma `(5, 3)` y `X` tiene forma `7`, no podemos multiplicarlos y, de hecho, este es el error al que nos estamos enfrentando:

In [ ]:
m = tfd.Normal(0., 1.).sample([5, 3])
try:
  m * X
except tf.errors.InvalidArgumentError as e:
  print(e)

Incompatible shapes: [5,3] vs. [7] [Op:Mul]


Para resolver este problema, pensemos en qué propiedades debe tener la distribución sobre `Y`. Si hemos llamado `jds_i.sample([5, 3])`, entonces sabemos que `m` y `b` tendrán la forma `(5, 3)`. ¿Qué forma debería producir una llamada a `sample` en la distribución `Y`? La respuesta obvia es `(5, 3, 7)`: para cada punto del lote, queremos una muestra del mismo tamaño que `X`. Podemos lograr esto mediante el uso de las capacidades de difusión de TensorFlow, agregando dimensiones adicionales:

In [ ]:
m[..., tf.newaxis].shape

TensorShape([5, 3, 1])

In [ ]:
(m[..., tf.newaxis] * X).shape

TensorShape([5, 3, 7])

Al agregar un eje tanto a `m` como a `b`, podemos definir un nuevo JDS que admita múltiples muestras:

In [ ]:
jds_ia = tfd.JointDistributionSequential([
    tfd.Normal(loc=0., scale=1.),   # m
    tfd.Normal(loc=0., scale=1.),   # b
    lambda b, m: tfd.Independent(   # Y
        tfd.Normal(loc=m[..., tf.newaxis]*X + b[..., tf.newaxis], scale=1.),
        reinterpreted_batch_ndims=1)
])

shaped_sample = jds_ia.sample([5, 3])
shaped_sample

[<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
 array([[-1.1133379 ,  0.16390413, -0.24177533],
        [-1.1312429 , -0.6224666 , -1.8182136 ],
        [-0.31343174, -0.32932565,  0.5164407 ],
        [-0.0119963 , -0.9079621 ,  2.3655841 ],
        [-0.26293617,  0.8229698 ,  0.31098196]], dtype=float32)>,
 <tf.Tensor: shape=(5, 3), dtype=float32, numpy=
 array([[-0.02876974,  1.0872147 ,  1.0138507 ],
        [ 0.27367726, -1.331534  , -0.09084719],
        [ 1.3349475 , -0.68765205,  1.680652  ],
        [ 0.75436825,  1.3050154 , -0.9415123 ],
        [-1.2502679 , -0.25730947,  0.74611956]], dtype=float32)>,
 <tf.Tensor: shape=(5, 3, 7), dtype=float32, numpy=
 array([[[-1.8258233e+00, -3.0641669e-01, -2.7595463e+00, -1.6952467e+00,
          -4.8197951e+00, -5.2986512e+00, -6.6931367e+00],
         [ 3.6438566e-01,  1.0067395e+00,  1.4542470e+00,  8.1155670e-01,
           1.8868095e+00,  2.3877139e+00,  1.0195159e+00],
         [-8.3624744e-01,  1.2518480e+00,  1.0943471e+00, 

In [ ]:
jds_ia.log_prob(shaped_sample)

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[-12.483114 , -10.139662 , -11.514159 ],
       [-11.656767 , -17.201958 , -12.132455 ],
       [-17.838818 ,  -9.474525 , -11.24898  ],
       [-13.95219  , -12.490049 , -17.123957 ],
       [-14.487818 , -11.3755455, -10.576363 ]], dtype=float32)>

Como verificación adicional, verificaremos que la probabilidad logarítmica para un único punto del lote coincida con la que teníamos antes:

In [ ]:
(jds_ia.log_prob(shaped_sample)[3, 1] -
 jds_i.log_prob([shaped_sample[0][3, 1],
                 shaped_sample[1][3, 1],
                 shaped_sample[2][3, 1, :]]))

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

<a id="AutoBatching-For-The-Win"></a>

### Procesamiento automático por lotes para ganar


¡Excelente! Ahora tenemos una versión de JointDistribution que maneja todos nuestros desiderata: `log_prob` devuelve un escalar gracias al uso de `tfd.Independent`, y las muestras múltiples funcionan ahora que hemos arreglado la difusión mediante la adición de ejes adicionales.

¿Qué pasaría si le dijera que hay una manera que es mejor y más fácil? Existe y se llama `JointDistributionSequentialAutoBatched` (JDSAB):

In [ ]:
jds_ab = tfd.JointDistributionSequentialAutoBatched([
    tfd.Normal(loc=0., scale=1.),   # m
    tfd.Normal(loc=0., scale=1.),   # b
    lambda b, m: tfd.Normal(loc=m*X + b, scale=1.) # Y
])

In [ ]:
jds_ab.log_prob(jds.sample())

<tf.Tensor: shape=(), dtype=float32, numpy=-12.954952>

In [ ]:
shaped_sample = jds_ab.sample([5, 3])
jds_ab.log_prob(shaped_sample)

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[-12.191533 , -10.43885  , -16.371655 ],
       [-13.292994 , -11.97949  , -16.788685 ],
       [-15.987699 , -13.435732 , -10.6029   ],
       [-10.184758 , -11.969714 , -14.275676 ],
       [-12.740775 , -11.5654125, -12.990162 ]], dtype=float32)>

In [ ]:
jds_ab.log_prob(shaped_sample) - jds_ia.log_prob(shaped_sample)

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32)>

¿Cómo funciona esto? Si bien puede tratar de [leer el código](https://github.com/tensorflow/probability/blob/main/tensorflow_probability/python/distributions/joint_distribution_auto_batched.py#L426) para comprenderlo en profundidad, le brindaremos una breve descripción general que es suficiente para la mayoría de los casos de uso:

- Recuerde que nuestro primer problema fue que nuestra distribución para `Y` tenía `batch_shape=[7]` y `event_shape=[]` y usamos `Independent` para convertir la dimensión de lote en una dimensión de evento. JDSAB ignora las formas por lotes de las distribuciones de componentes; en lugar de eso, trata la forma de lote como una propiedad general del modelo, que se supone que es `[]` (a menos que se especifique lo contrario al establecer `batch_ndims > 0`). El efecto es equivalente al uso de tfd.Independent para convertir *todas* las dimensiones de lote de distribuciones de componentes en dimensiones de eventos, como lo hicimos manualmente anteriormente.
- Nuestro segundo problema fue la necesidad de manipular las formas de `m` y `b` para que pudieran difundirse correctamente con `X` al crear múltiples muestras. Con JDSAB, usted escribe un modelo para generar una sola muestra y nosotros "levantamos" todo el modelo para generar múltiples muestras con ayuda de [vectorized_map](https://www.tensorflow.org/api_docs/python/tf/vectorized_map) de TensorFlow. (Esta característica es análoga al [vmap](https://jax.readthedocs.io/en/latest/notebooks/quickstart.html#Auto-vectorization-with-vmap) de JAX).

Al explorar el problema de la forma de lote con más detalle, podemos comparar las formas de lote de nuestra distribución conjunta "mala" original `jds`, nuestras distribuciones fijadas por lotes `jds_i` y `jds_ia`, y nuestro `jds_ab` con lotes automáticos:

In [ ]:
jds.batch_shape

[TensorShape([]), TensorShape([]), TensorShape([7])]

In [ ]:
jds_i.batch_shape

[TensorShape([]), TensorShape([]), TensorShape([])]

In [ ]:
jds_ia.batch_shape

[TensorShape([]), TensorShape([]), TensorShape([])]

In [ ]:
jds_ab.batch_shape

TensorShape([])

Vemos que el `jds` original tiene subdistribuciones con diferentes formas de lote. `jds_i` y `jds_ia` solucionan este problema mediante la creación de subdistribuciones con la misma forma de lote (vacía). `jds_ab` tiene solo una forma de lote única (vacía).

Vale la pena señalar que `JointDistributionSequentialAutoBatched` ofrece generalidad adicional de forma gratuita. Supongamos que hacemos que las covariables `X` (y, de forma implícita, las observaciones `Y`) sean bidimensionales:

In [ ]:
X = np.arange(14).reshape((2, 7))
X

array([[ 0,  1,  2,  3,  4,  5,  6],
       [ 7,  8,  9, 10, 11, 12, 13]])

Nuestro `JointDistributionSequentialAutoBatched` funciona sin cambios (necesitamos redefinir el modelo porque `jds_ab.log_prob` almacena en caché la forma de `X`):

In [ ]:
jds_ab = tfd.JointDistributionSequentialAutoBatched([
    tfd.Normal(loc=0., scale=1.),   # m
    tfd.Normal(loc=0., scale=1.),   # b
    lambda b, m: tfd.Normal(loc=m*X + b, scale=1.) # Y
])

shaped_sample = jds_ab.sample([5, 3])
shaped_sample

[<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
 array([[ 0.1813647 , -0.85994506,  0.27593774],
        [-0.73323774,  1.1153806 ,  0.8841938 ],
        [ 0.5127983 , -0.29271227,  0.63733214],
        [ 0.2362284 , -0.919168  ,  1.6648189 ],
        [ 0.26317367,  0.73077047,  2.5395133 ]], dtype=float32)>,
 <tf.Tensor: shape=(5, 3), dtype=float32, numpy=
 array([[ 0.09636458,  2.0138032 , -0.5054413 ],
        [ 0.63941646, -1.0785882 , -0.6442188 ],
        [ 1.2310615 , -0.3293852 ,  0.77637213],
        [ 1.2115169 , -0.98906034, -0.07816773],
        [-1.1318136 ,  0.510014  ,  1.036522  ]], dtype=float32)>,
 <tf.Tensor: shape=(5, 3, 2, 7), dtype=float32, numpy=
 array([[[[-1.9685398e+00, -1.6832136e+00, -6.9127172e-01,
            8.5992378e-01, -5.3123581e-01,  3.1584005e+00,
            2.9044402e+00],
          [-2.5645006e-01,  3.1554163e-01,  3.1186538e+00,
            1.4272424e+00,  1.2843871e+00,  1.2266440e+00,
            1.2798605e+00]],
 
         [[ 1.5973477e+00,

In [ ]:
jds_ab.log_prob(shaped_sample)

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[-28.90071 , -23.052422, -19.851362],
       [-19.775568, -25.894997, -20.302256],
       [-21.10754 , -23.667885, -20.973007],
       [-19.249458, -20.87892 , -20.573763],
       [-22.351208, -25.457762, -24.648403]], dtype=float32)>

Por otro lado, nuestro `JointDistributionSequential` cuidadosamente elaborado ya no funciona:

In [ ]:
jds_ia = tfd.JointDistributionSequential([
    tfd.Normal(loc=0., scale=1.),   # m
    tfd.Normal(loc=0., scale=1.),   # b
    lambda b, m: tfd.Independent(   # Y
        tfd.Normal(loc=m[..., tf.newaxis]*X + b[..., tf.newaxis], scale=1.),
        reinterpreted_batch_ndims=1)
])

try:
  jds_ia.sample([5, 3])
except tf.errors.InvalidArgumentError as e:
  print(e)

Incompatible shapes: [5,3,1] vs. [2,7] [Op:Mul]


Para solucionar este problema, tendríamos que agregar un segundo `tf.newaxis` para que `m` y `b` coincidan con la forma, y ​​aumentar `reinterpreted_batch_ndims` a 2 en la llamada a `Independent`. En este caso, dejar que la maquinaria de procesamiento automático por lotes se encargue de los problemas de forma es más rápido, más fácil y más ergonómico.

Una vez más, observamos que si bien este bloc de notas exploró `JointDistributionSequentialAutoBatched`, las otras variantes de `JointDistribution` tienen `AutoBatched` equivalente. (Para los usuarios de `JointDistributionCoroutine`, `JointDistributionCoroutineAutoBatched` tiene el beneficio adicional de que ya no necesita especificar nodos `Root`; si nunca ha usado `JointDistributionCoroutine`, puede ignorar esta instrucción con seguridad).

### Reflexiones finales

En este bloc de notas, presentamos `JointDistributionSequentialAutoBatched` y trabajamos en detalle con un ejemplo simple. ¡Esperamos que haya aprendido algo sobre las formas de TFP y sobre el procesamiento automático por lotes!